In [ ]:
pip install torch torchvision matplotlib opencv-python ultralytics


In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

# Load the YOLOv5 model
model = YOLO("yolov5s.pt")  # YOLOv5 small model

# Load an image
image_path = "sample_image.jpg"
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Perform object detection
results = model(image)

# Visualize the results
results.render()
plt.imshow(results.imgs[0])
plt.axis("off")
plt.title("Object Detection with YOLOv5")
plt.show()


In [ ]:
# Initialize webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Perform object detection
    results = model(frame)
    
    # Display the results
    results.render()
    cv2.imshow("YOLOv5 Object Detection", results.imgs[0])
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import torch
from torch import nn
from torchvision.transforms import ToTensor, Resize
from torchvision.datasets import VOCSegmentation
from torch.utils.data import DataLoader

# Define the U-Net model
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.decoder = nn.Sequential(
            nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Initialize the U-Net model
unet_model = UNet()
unet_model = unet_model.train()


In [ ]:
# Load the dataset
dataset = VOCSegmentation(root="./data", year="2012", image_set="train", download=True, transform=ToTensor())
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Display a sample image and mask
image, mask = dataset[0]
plt.subplot(1, 2, 1)
plt.imshow(image.permute(1, 2, 0))
plt.title("Image")
plt.subplot(1, 2, 2)
plt.imshow(mask, cmap="gray")
plt.title("Mask")
plt.show()


In [ ]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(unet_model.parameters(), lr=1e-3)

# Training loop
epochs = 3
for epoch in range(epochs):
    total_loss = 0
    for images, masks in dataloader:
        optimizer.zero_grad()
        predictions = unet_model(images)
        loss = criterion(predictions, masks)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(dataloader):.4f}")


In [ ]:
# Test the model
unet_model.eval()
with torch.no_grad():
    test_image, test_mask = dataset[10]
    test_image = test_image.unsqueeze(0)
    output = unet_model(test_image)

    # Convert to binary mask
    output_mask = torch.argmax(output.squeeze(), dim=0)

# Visualize results
plt.subplot(1, 2, 1)
plt.imshow(test_image.squeeze().permute(1, 2, 0))
plt.title("Input Image")
plt.subplot(1, 2, 2)
plt.imshow(output_mask, cmap="gray")
plt.title("Predicted Mask")
plt.show()
